Use this utility to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['SEC filings', 'Headquarters Location', 'Date first added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,Industrials,Industrial Conglomerates
AOS,A. O. Smith,Industrials,Building Products
ABT,Abbott,Health Care,Health Care Equipment
ABBV,AbbVie,Health Care,Pharmaceuticals
ABMD,Abiomed,Health Care,Health Care Equipment
...,...,...,...
YUM,Yum! Brands,Consumer Discretionary,Restaurants
ZBRA,Zebra,Information Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,Health Care,Health Care Equipment


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp500 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp500.index = sp500.index.str.replace('.', '-', regex=False)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,US Stocks:Industrials,Industrial Conglomerates
AOS,A. O. Smith,US Stocks:Industrials,Building Products
ABT,Abbott,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie,US Stocks:Healthcare,Pharmaceuticals
ABMD,Abiomed,US Stocks:Healthcare,Health Care Equipment
...,...,...,...
YUM,Yum! Brands,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra,US Stocks:Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,US Stocks:Healthcare,Health Care Equipment


In [8]:
# Make symbols list.
symbols = list(sp500.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2022-04-18,217.83,372.42,425.47,497.53,93.89,100.58,165.07,280.52,275.07,572.13,3055.70,480.63,265.39,723.20,34.29,203.26,130.76,150.61,187.01,478.01
2022-04-19,221.98,371.00,436.40,514.40,96.93,103.66,167.40,285.30,273.10,574.15,3162.31,491.79,273.82,716.22,34.59,208.81,132.84,155.62,191.39,490.52
2022-04-20,214.82,371.75,428.11,505.22,94.02,94.90,167.23,286.36,280.38,591.15,3079.96,496.39,276.90,719.62,34.59,209.19,132.70,157.13,186.23,493.54
2022-04-21,201.83,365.14,417.48,491.67,89.85,89.40,166.42,280.81,278.24,580.43,2965.92,472.41,275.08,706.11,34.51,206.86,132.10,151.86,177.23,474.78
2022-04-22,195.15,349.77,408.67,471.40,88.14,86.03,161.79,274.03,264.56,561.28,2887.00,440.65,269.03,688.08,33.58,205.21,128.63,148.07,171.43,456.04


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

-23.77715651825829

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.16749414736546026

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.0859583540497838

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2021-12-31    0.04
2022-01-31   -0.07
2022-02-28   -0.10
2022-03-31    0.00
2022-04-30    0.00
Freq: M, Name: MMM, Length: 88, dtype: float64

In [18]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.20809562082464841

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')